# ドンチャンチャンネルを用いたスウィングトレードのデモ

## ライブラリのインポート

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from pandas_datareader import data as pdr
import pandas_ta as ta

## データ獲得関数の定義

In [2]:
def yf_obtain(symbol, ndays):
    end_date = datetime.today()
    start_date = end_date - timedelta(days=ndays)
    yf.pdr_override()
    data = pdr.get_data_yahoo(symbol, start_date, end_date)
    return data

## トレード関数の定義

In [3]:
def trade_donchian(row):
    global trades, trade_open
    row = row.to_dict()

    if((trade_open==True) and (row['long'] == 1)): pass

    elif((trade_open==False) and (row['short'] == 1)): pass

    elif((trade_open==False) and (row['long'] == 1)):
        # open trade
        trade_open = True
        _trade = {
            'buy_date': row['next_date'],
            'buy_price': round(row['next_day_open_price']*1.005,2),
            'sell_price': None,
            'sell_date': None,
        }
        trades.append(_trade)
        del _trade

    elif((trade_open==True) and (row['short'] == 1)):
        # close trade
        trade_open = False
        _trade = trades[-1]
        _trade['sell_date'] = row['next_date']
        _trade['sell_price'] = round(row['next_day_open_price']*0.995,2)
        trades[-1] = _trade
        del _trade


## バックテスト関数の定義

In [4]:
def backtest(symbol, ndays, low=20, high=20):
    global trades, trade_open
    
    # get the stock prices
    data = yf_obtain(symbol, ndays)
    
    data = data.reset_index()
    
    # calculate donchian channels
    data[['low', 'mid', 'high']] = data.ta.donchian(lower_length=low, upper_length=high)
    
    # implement the trading strategy
    data['long'] = ((data['Close']==data['low'])|(data['Low']==data['low'])).astype('int')
    data['short'] = ((data['Close']==data['high'])|(data['High']==data['high'])).astype('int')
     
    # get the next day open price and date
    data['next_day_open_price'] = data['Open'].shift(-1)
    data['next_date'] = data['Date'].shift(-1).astype('string')
    
    trade_open = False
    trades = []
    data.dropna(inplace=True)
    
    cols = ['Date', 'Open', 'Close', 'Adj Close', 'Low', 'High', 
            'low', 'mid', 'high', 'long', 'short', 'next_day_open_price', 
            'next_date']

    data = data[cols]

    data.apply(trade_donchian, axis=1)

    if(len(trades)==0): return None

    x = pd.DataFrame(trades)
    
    # calculate the returns and holding period
    x['buy_date'] = pd.to_datetime(x['buy_date'], format="%Y-%m-%d", dayfirst=True)
    x['sell_date'] = pd.to_datetime(x['sell_date'], format="%Y-%m-%d", dayfirst=True)
    x['returns'] = round(100*(x['sell_price']-x['buy_price'])/x['buy_price'],2)
    x['holding_period'] = (x['sell_date'] - x['buy_date']).dt.days
    x['stock'] = stock
    return x


## 株価データによる評価

In [5]:
TRADES = pd.DataFrame()
trades = []
trade_open = False

# 日本の時価総額上位銘柄
Japanese_stocks = ['7203.T','6758.T','9432.T','6861.T','8306.T',
                   '8035.T','4063.T','9983.T','9433.T','6098.T',
                   '8058.T','4661.T','6501.T','9984.T','7974.T',
                   '8001.T','8316.T','4519.T','9434.T','8031.T',
                   '7267.T','4568.T','2914.T','8766.T','6367.T',
                   '6902.T','4502.T','7741.T','8411.T','6981.T',
                   '7182.T','6273.T','6723.T','3382.T','7751.T',
                   '6702.T','6178.T','6503.T','6954.T','5108.T',
                   '4689.T','8053.T','8002.T','6146.T','6201.T',
                   '6857.T','9022.T','6301.T','6920.T','4901.T']
                   

for stock in Japanese_stocks:
    _tr = backtest(stock, 1800, 20, 20)
    if(len(TRADES)==0): TRADES = _tr
    else: TRADES = pd.concat([TRADES, _tr], ignore_index=True)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [6]:
TRADES.head()

,buy_date,buy_price,sell_price,sell_date,returns,holding_period,stock
0,2019-03-29,1319.36,1349.02,2019-04-05,2.25,7.0,7203.T
1,2019-05-10,1320.97,1336.29,2019-06-11,1.16,32.0,7203.T
2,2019-08-06,1333.63,1387.63,2019-09-03,4.05,28.0,7203.T
3,2020-01-07,1538.45,1562.15,2020-01-22,1.54,15.0,7203.T
4,2020-02-05,1549.71,1583.84,2020-02-07,2.20,2.0,7203.T
